In [1]:
import networkx as nx
import csv
import operator

For now, we're only looking at the first set of tweets.
# Get Data from .csv

In [2]:
csv_file = open('all_tweets/IRAhandle_tweets_1.csv') # change this to whatever group you wanna look at
data = list(csv.DictReader(csv_file))

In [3]:
print(len(data))

243891


Let's take a look at the domains that these tweets are linking to.
# Get Domain Info & Sort Into All / Russian / Non-Russian
##### Format of each dictionary:

{(string) *domain_name*: (array) [(int) *count*, (array) *tweet_id's* linking to this domain]}

In [4]:
link_categories = ['tco1_step1', 'tco2_step1', 'tco3_step1']
user_info = ['author']
details = ['retweet', 'account_category']

In [5]:
domain_starters = ['http://', 'https://']
common_bits = ['www.', '.com']

def get_domain_old(link, no_common = False):
    # removes the http:// and https://, and does not have the ending slash
    longest_starter_len = max([len(i) for i in domain_starters])
    longest_common_len = max([len(i) for i in common_bits])
    for ds in domain_starters:
        if ds in link[:longest_starter_len]:
            beginning = link[len(ds):].split('/')[0]
            if no_common:
                for b in common_bits:
                    if b in beginning:
                        beginning = [r for r in beginning.split(b) if r != ''][0] 
            return beginning.lower()

def get_domain(link, no_common = False):
    longest_common_len = max([len(i) for i in common_bits])
    beginning = '/'.join(link.split('/')[:3]) # we assume we have something like pre://domain_name/...
    if no_common:
        for b in common_bits:
            if b in beginning:
                beginning = [r for r in beginning.split(b) if r != ''][0]
    return beginning.lower() + '/'
        
def basic_dict_checks(dct, key, twt_id):
    if key in dct.keys():
        dct[key][COUNT_IND] += 1
        dct[key][TWT_ID_IND] += [twt_id]
    else:
        dct[key] = []
        dct[key] += [1]
        dct[key] += [[twt_id]]
print(get_domain("http://www.youtu.be/hiwww"))

http://www.youtu.be/


In [25]:
unique_domains = {}
russian_sites = {}
non_russian_sites = {}

COUNT_IND = 0
TWT_ID_IND = 1

total_links = 0

for tweet in data:
    tweet_id = tweet['tweet_id']
    for link in link_categories:
        dom = get_domain(tweet[link])
        if dom and dom != '/':
            basic_dict_checks(unique_domains, dom, tweet_id)
            if '.ru' in dom:
                basic_dict_checks(russian_sites, dom, tweet_id)
            else:
                basic_dict_checks(non_russian_sites, dom, tweet_id)
            total_links += 1

### Sort the Domains (now stored in lists) by Popularity

In [7]:
sorted_domains = sorted(unique_domains.items(), key=lambda k: k[1][0], reverse=True)
sorted_russian_sites = sorted(russian_sites.items(), key=lambda k: k[1][0], reverse=True)
sorted_non_russian_sites = sorted(non_russian_sites.items(), key=lambda k: k[1][0], reverse=True)

In [8]:
def domain_basics(site_info):
    # given a site: info pair, returns an array with the domain name and number of tweets linking to that domain
    return [site_info[0], site_info[1][COUNT_IND]]
def print_title(title):
    print(title)
    print("========================================")

In [9]:
print_title("Top 20 Referenced Sites in Russian Tweets (Ignoring Twitter)")
for site in sorted_domains[1:21]:
    if '.ru' in site[0]:
        print('\nRUSSIAN: ' + str(domain_basics(site)) + '\n')
    else:
        print(domain_basics(site))

Top 20 Referenced Sites in Russian Tweets (Ignoring Twitter)
['http://ift.tt/', 24008]

RUSSIAN: ['http://www.gazeta.ru/', 7728]

['http://bit.ly/', 7552]
['http://dlvr.it/', 3794]

RUSSIAN: ['http://ria.ru/', 3548]

['https://twibble.io/', 3427]
['http://twib.in/', 3122]

RUSSIAN: ['http://tass.ru/', 2921]

['http://www.fox5atlanta.com/', 2706]
['http://ow.ly/', 2349]
['http://usfreedomarmy.com/', 2070]
['http://fb.me/', 1668]
['http://www.kob.com/', 1615]
['https://www.youtube.com/', 1494]
['https://youtu.be/', 1258]
['https://goo.gl/', 1256]
['https://www.instagram.com/', 1220]
['http://krqe.com/', 1152]

RUSSIAN: ['http://www.rbc.ru/', 1128]

['http://dld.bz/', 965]


In [10]:
print_title("Top 20 Russian Sites in Russian Tweets")
for site in sorted_russian_sites[:20]:
    print(domain_basics(site))

Top 20 Russian Sites in Russian Tweets
['http://www.gazeta.ru/', 7728]
['http://ria.ru/', 3548]
['http://tass.ru/', 2921]
['http://www.rbc.ru/', 1128]
['http://www.postsovet.ru/', 942]
['http://riafan.ru/', 561]
['http://r.rbc.ru/', 486]
['http://rsport.ru/', 413]
['https://ria.ru/', 397]
['http://vesti.ru/', 323]
['http://www.vesti.ru/', 310]
['http://lifenews.ru/', 291]
['https://tvrain.ru/', 286]
['http://www.kp.ru/', 268]
['http://izvestia.ru/', 244]
['http://top.rbc.ru/', 210]
['http://vz.ru/', 201]
['http://tvzvezda.ru/', 200]
['http://go.tass.ru/', 194]
['http://www.ntv.ru/', 148]


In [11]:
print_title("Top 20 Non-Russian Sites in Russian Tweets")
for site in sorted_non_russian_sites[:20]:
    print(domain_basics(site))

Top 20 Non-Russian Sites in Russian Tweets
['https://twitter.com/', 121457]
['http://ift.tt/', 24008]
['http://bit.ly/', 7552]
['http://dlvr.it/', 3794]
['https://twibble.io/', 3427]
['http://twib.in/', 3122]
['http://www.fox5atlanta.com/', 2706]
['http://ow.ly/', 2349]
['http://usfreedomarmy.com/', 2070]
['http://fb.me/', 1668]
['http://www.kob.com/', 1615]
['https://www.youtube.com/', 1494]
['https://youtu.be/', 1258]
['https://goo.gl/', 1256]
['https://www.instagram.com/', 1220]
['http://krqe.com/', 1152]
['http://dld.bz/', 965]
['https://russian.rt.com/', 943]
['http://ind.pn/', 904]
['http://larep.it/', 843]


In [56]:
t_russites = len(sorted_russian_sites)
t_norussites = len(sorted_non_russian_sites)
t_sites = len(sorted_domains)
total_tweets = len(data)

t_ruslinks = sum([deet[0] for (name, deet) in sorted_russian_sites])
t_noruslinks = sum([deet[0] for (name, deet) in sorted_non_russian_sites])

print_title("Numbers of Unique Domains:")
print('Russian: ' + str(t_russites))
print('Non-Russian: ' + str(t_norussites))
print('for a total of ' + str(t_sites) + ' unique domains for ' + str(total_tweets) + ' tweets')
print('percentage of unique Russian sites over all: ' + str(round(t_russites / t_sites * 100, 2)) + '%')

print_title("\nNumber of Links to Domains:")
print('Russian: ' + str(t_ruslinks))
print('Non-Russian: ' + str(t_noruslinks))
print('for a total of ' + str(total_links) + ' links in ' + str(total_tweets) + ' tweets')
print('percentage of Russian links over all: ' + str(round(t_ruslinks / total_links * 100, 2)) + '%')

Numbers of Unique Domains:
Russian: 479
Non-Russian: 5756
for a total of 6235 unique domains for 243891 tweets
percentage of unique Russian sites over all: 7.68%

Number of Links to Domains:
Russian: 23240
Non-Russian: 226515
for a total of 249755 links in 243891 tweets
percentage of Russian links over all: 9.31%


# Some Preliminary Observations:
### Some automation websites
dlvr.it, twibble.io + twib.in, ift.tt (?)
### A lot of shortening websites
tinyurl, bit.ly, ow.ly, goo.gl, fb.me
### A few repeated websites w/ 'different' domains
youtu.be vs youtube.com, rbc.com
### Top sites are:
shortening websites, automation websites, social media, Fox news, USFreedomArmy; RUSSIATODAY (not included in Russian websites right now)
# NEXT question: matching with our categories?
## NOTE: this is all without having figured out all shortlink websites thus far; numbers may (?) go up afterwards

In [98]:
def read_and_match_sites(types):
    # takes a list of category names
    sites = []
    for name in types:
        sites += [i.rstrip('\n') for i in open('categories/' + name).readlines()]
    matching = []
    for site in sites:
        if site in unique_domains.keys():
            matching += [(site, unique_domains[site][0])]
    return matching

def total_matches(matches):
    return sum([num for name, num in matches])

In [95]:
fake_matches = read_and_match_sites(['fake-news'])

In [111]:
print_title('Top 20 Matching Fake News Sites & Counts')
for i in sorted(fake_matches, key = operator.itemgetter(1), reverse=True)[:20]:
    print(i)
print('\nTotal Fake link matches: ' + str(total_matches(fake_matches)) + ' out of ' + str(total_links) + ' total links')
print('Percentage fake over all: ' + str(round(total_matches(fake_matches) / total_links * 100, 3)) + ' %')

Top 20 Matching Fake News Sites & Counts
('http://www.thegatewaypundit.com/', 307)
('http://conservativetribune.com/', 122)
('http://bb4sp.com/', 58)
('http://www.dcclothesline.com/', 49)
('http://beforeitsnews.com/', 43)
('http://eaglerising.com/', 43)
('http://100percentfedup.com/', 31)
('https://www.jihadwatch.org/', 27)
('http://www.truthrevolt.org/', 22)
('https://conservativedailypost.com/', 20)
('http://conservativefighters.com/', 20)
('http://pamelageller.com/', 19)
('http://rickwells.us/', 15)
('http://www.frontpagemag.com/', 15)
('http://freedomdaily.com/', 13)
('http://endingthefed.com/', 11)
('http://www.israelvideonetwork.com/', 11)
('http://www.barenakedislam.com/', 10)
('http://freedomoutpost.com/', 10)
('http://allenwestrepublic.com/', 8)

Total Fake link matches: 1034 out of 249755 total links
Percentage fake over all: 0.414 %


In [94]:
conspiracy_matches = read_and_match_sites(['conspiracy'])

In [109]:
print_title('Top 20 Matching Conspiracy Sites & Counts')
for i in sorted(conspiracy_matches, key = operator.itemgetter(1), reverse=True)[:20]:
    print(i)
print('\nTotal Conspiracy link matches: ' + str(total_matches(conspiracy_matches)) + ' out of ' + str(total_links) + ' total_links')
print('Percentage conspiracy over all: ' + str(round(total_matches(conspiracy_matches) / total_links * 100, 3)) + ' %')

Top 20 Matching Conspiracy Sites & Counts
('http://www.zerohedge.com/', 212)
('http://www.infowars.com/', 116)
('http://truepundit.com/', 71)
('http://worldtruth.tv/', 29)
('http://consciouslifenews.com/', 21)
('http://www.disclose.tv/', 21)
('http://yournewswire.com/', 20)
('http://21stcenturywire.com/', 18)
('http://countercurrentnews.com/', 17)
('http://www.veteranstoday.com/', 14)
('http://shoebat.com/', 12)
('http://www.activistpost.com/', 11)
('http://www.nowtheendbegins.com/', 10)
('http://www.ecowatch.com/', 7)
('http://www.globalresearch.ca/', 7)
('http://wearechange.org/', 7)
('http://theantimedia.org/', 7)
('http://www.coasttocoastam.com/', 6)
('https://www.sott.net/', 6)
('http://www.centerforsecuritypolicy.org/', 5)

Total Conspiracy link matches: 675 out of 249755 total_links
Percentage conspiracy over all: 0.27 %
